# Sprawozdanie

In [1]:
# ============================================================
# SEKCJA 1: Import bibliotek i funkcje pomocnicze
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Funkcja: Rozwiązanie równania Bellmana dla polityki
def evaluate_policy_linear_system(P_pi: np.ndarray, r_pi: np.ndarray, gamma: float) -> np.ndarray:
    """
    Rozwiązuje równanie Bellmana dla danej polityki w postaci macierzowej.
    
    Dla polityki π mamy:
        v = r_pi + gamma * P_pi * v
    czyli:
        (I - gamma * P_pi) v = r_pi
    
    Parametry:
    -----------
    P_pi : ndarray (nS x nS)
        Macierz przejść dla polityki π
    r_pi : ndarray (nS,)
        Wektor nagród oczekiwanych dla polityki π
    gamma : float
        Współczynnik dyskontowania
    
    Zwraca:
    -------
    v : ndarray (nS,)
        Funkcja wartości dla polityki π
    """
    nS = P_pi.shape[0]
    I = np.eye(nS)
    return np.linalg.solve(I - gamma * P_pi, r_pi)

# Funkcja pomocnicza: Wyświetlenie wartości jako siatkę
def pretty_matrix_as_grid(v: np.ndarray, nrow: int, ncol: int, decimals: int = 1):
    """Wyświetl wektor wartości jako siatkę (nrow x ncol)."""
    grid = v.reshape(nrow, ncol)
    with np.printoptions(precision=decimals, suppress=True):
        print(grid)

# Funkcja pomocnicza: Wizualizacja polityki jako strzałki
def action_arrows(pi_det: np.ndarray, nrow: int, ncol: int):
    """Zamienia deterministyczną politykę na strzałki w siatce."""
    arrows = {0:'↑', 1:'→', 2:'↓', 3:'←', None:'·'}
    out = []
    for r in range(nrow):
        row = []
        for c in range(ncol):
            s = r*ncol + c
            a = int(pi_det[s]) if pi_det[s] is not None else None
            row.append(arrows.get(a, '?'))
        out.append(' '.join(row))
    print('\n'.join(out))

print("✓ Biblioteki i funkcje pomocnicze załadowane")

✓ Biblioteki i funkcje pomocnicze załadowane


---

## Sekcja 2: Komponenty MDP - Środowisko vs Agent

### Trzy kluczowe komponenty MDP:

#### 1. **Model Środowiska (Environment Model) - P[s][a]**
- Definiuje dynamikę świata
- Zawiera informacje o przejściach między stanami i nagrodach
- Format: `P[s][a] → [(p, s', r, terminated), ...]`
  - `p`: prawdopodobieństwo przejścia
  - `s'`: następny stan
  - `r`: nagroda
  - `terminated`: czy epizod się kończy

**Przykład**: Recycling Robot zdefiniowany parametrami:
- `α` (alpha): prawdopodobieństwo pozostania w stanie H po SEARCH
- `β` (beta): prawdopodobieństwo pozostania w stanie L po SEARCH
- `r_search`, `r_wait`: nagrody za akcje
- `rescue_cost`: kara za rozładowanie

#### 2. **Polityka Agenta (Policy) - π[s,a]**
- Definiuje zachowanie agenta
- Macierz rozmiar: `(nStanów, nAkcji)`
- `π[s,a]` = prawdopodobieństwo wybrania akcji `a` w stanie `s`
- Dla polityki deterministycznej: `π[s,a] ∈ {0, 1}`

**Przykład**: π(H)=SEARCH, π(L)=RECHARGE oznacza:
- W stanie H zawsze szukaj (SEARCH)
- W stanie L zawsze ładuj (RECHARGE)

#### 3. **Funkcja Wartości (Value Function) - v_π[s]**
- Mierzy oczekiwany zdyskontowany zwrot
- `v_π(s) = E_π[∑_{t=0}^∞ γ^t R_{t+1} | S_t = s]`
- Odpowiada na pytanie: "Ile średnio zyskamy w przyszłości startując ze stanu s i stosując politykę π?"

### Relacja między komponentami:

```
┌─────────────────────────────────────┐
│  ŚRODOWISKO (Environment)           │
│  Model P[s][a]                      │
│  - przejścia między stanami          │
│  - nagrody                           │
│  (definiuje możliwości świata)       │
└──────────────┬──────────────────────┘
               │
               ↓ (Agent w nim funkcjonuje)
               
┌──────────────────────────────────────┐
│  AGENT                               │
│  Polityka π[s,a]                     │
│  - decyzje w każdym stanie           │
│  (definiuje sposób działania agenta) │
└──────────────┬──────────────────────┘
               │
               ↓ (Razem dają wynik)
               
┌──────────────────────────────────────┐
│  Funkcja Wartości v_π[s]             │
│  - oczekiwane zwroty                 │
│  (ocena jakości polityki)            │
└──────────────────────────────────────┘
```

**Wniosek**: Parametry świata (α, β, rescue_cost, gamma) **NIE są polityką**. Ale wpływają na wartości akcji, co zmienia **która polityka jest optymalna**.

---

## Sekcja 3: Model Recycling Robot - Budowa P[s][a]

### Opis problemu

**Recycling Robot** to prosty MDP z 2 stanami reprezentującymi poziom energii baterii:
- **Stan H** (High): wysoka energia
- **Stan L** (Low): niska energia

**Dostępne akcje:**
1. **SEARCH (0)**: szuka puszek (nagroda +5), ale może rozładować baterię
2. **WAIT (1)**: czeka na człowieka (nagroda +1), brak rozładowania
3. **RECHARGE (2)**: ładuje baterię (tylko w stanie L, nagroda 0)

### Dynamika przejść:

| Stan | Akcja | Następny stan | Prawdopodobieństwo | Nagroda |
|------|-------|----------------|-------------------|---------|
| H | SEARCH | H | α | r_search |
| H | SEARCH | L | 1-α | r_search |
| H | WAIT | H | 1.0 | r_wait |
| L | SEARCH | L | β | r_search |
| L | SEARCH | H | 1-β | rescue_cost |
| L | WAIT | L | 1.0 | r_wait |
| L | RECHARGE | H | 1.0 | 0 |

In [2]:
def build_recycling_robot_P(alpha=0.8, beta=0.4, r_search=5.0, r_wait=1.0, rescue_cost=-3.0):
    """
    Buduje model MDP dla Recycling Robot.
    
    Parametry:
    ----------
    alpha : float
        Prawdopodobieństwo pozostania w H po SEARCH (domyślnie 0.8)
    beta : float
        Prawdopodobieństwo pozostania w L po SEARCH (domyślnie 0.4)
    r_search : float
        Nagroda za SEARCH (domyślnie 5.0)
    r_wait : float
        Nagroda za WAIT (domyślnie 1.0)
    rescue_cost : float
        Kara za rozładowanie w L (domyślnie -3.0)
    
    Zwraca:
    -------
    P : dict
        Model świata P[s][a] -> [(p, s', r, terminated), ...]
    nS : int
        Liczba stanów (2)
    nA : int
        Liczba akcji (3)
    """
    
    nS, nA = 2, 3  # 2 stany, 3 akcje
    P = {s: {a: [] for a in range(nA)} for s in range(nS)}
    
    # Aliasy dla czytelności
    H, L = 0, 1
    SEARCH, WAIT, RECHARGE = 0, 1, 2
    
    # ========== Stan H (wysoka energia) ==========
    
    # SEARCH w H: może pozostać w H (z p=alpha) lub przejść do L
    P[H][SEARCH] = [
        (alpha, H, r_search, False),
        (1 - alpha, L, r_search, False),
    ]
    
    # WAIT w H: zawsze pozostaje w H
    P[H][WAIT] = [
        (1.0, H, r_wait, False),
    ]
    
    # RECHARGE w H: akcja niedostępna
    P[H][RECHARGE] = []
    
    # ========== Stan L (niska energia) ==========
    
    # SEARCH w L: może pozostać w L (z p=beta) lub rozładować się i wrócić do H
    P[L][SEARCH] = [
        (beta, L, r_search, False),
        (1 - beta, H, rescue_cost, False),
    ]
    
    # WAIT w L: zawsze pozostaje w L
    P[L][WAIT] = [
        (1.0, L, r_wait, False),
    ]
    
    # RECHARGE w L: zawsze przechodzi do H z nagrodą 0
    P[L][RECHARGE] = [
        (1.0, H, 0.0, False),
    ]
    
    return P, nS, nA


# Test: Weryfikacja poprawności modelu
print("=" * 70)
print("TEST: Budowa modelu Recycling Robot")
print("=" * 70)

P, nS, nA = build_recycling_robot_P()

# Sprawdzenie rozmiarów
assert nS == 2 and nA == 3, "Zły rozmiar MDP"

# Sprawdzenie że prawdopodobieństwa sumują się do 1 dla każdej akcji
test_cases = [
    (0, 0, "H-SEARCH"),
    (0, 1, "H-WAIT"),
    (1, 0, "L-SEARCH"),
    (1, 1, "L-WAIT"),
    (1, 2, "L-RECHARGE"),
]

for s, a, name in test_cases:
    if P[s][a]:  # jeśli akcja jest dostępna
        prob_sum = sum(p for p, *_ in P[s][a])
        assert abs(prob_sum - 1.0) < 1e-12, f"Błąd w {name}: suma = {prob_sum}"
        print(f"  ✓ {name}: suma prawdopodobieństw = {prob_sum:.4f}")

# Sprawdzenie że RECHARGE w H jest niedostępna
assert P[0][2] == [], "RECHARGE w H powinien być niedostępny"
print(f"  ✓ H-RECHARGE: akcja niedostępna (pusta lista)")

print("\n✓ Wszystkie testy przeszły pomyślnie!")
print("\nModel zawiera:")
print(f"  - Liczba stanów (nS): {nS}")
print(f"  - Liczba akcji (nA): {nA}")
print(f"  - Razem możliwych par (s,a): {nS * nA}")

TEST: Budowa modelu Recycling Robot
  ✓ H-SEARCH: suma prawdopodobieństw = 1.0000
  ✓ H-WAIT: suma prawdopodobieństw = 1.0000
  ✓ L-SEARCH: suma prawdopodobieństw = 1.0000
  ✓ L-WAIT: suma prawdopodobieństw = 1.0000
  ✓ L-RECHARGE: suma prawdopodobieństw = 1.0000
  ✓ H-RECHARGE: akcja niedostępna (pusta lista)

✓ Wszystkie testy przeszły pomyślnie!

Model zawiera:
  - Liczba stanów (nS): 2
  - Liczba akcji (nA): 3
  - Razem możliwych par (s,a): 6


---

## Sekcja 4: Ewaluacja Polityki - A1.2

### Cel:
Dla zadanej polityki obliczyć funkcję wartości $v_\pi$ rozwiązując równanie Bellmana.

### Procedura:

1. **Budowa macierzy przejść i wektora nagród dla polityki**
   - Ze zwykłego modelu `P[s][a]` i polityki `π` konstruujemy `P_π` i `r_π`
   - Te obiekty reprezentują świat "widziany przez politykę"
   
2. **Rozwiązanie układu równań liniowych**
   - Równanie Bellmana: $v_\pi = r_\pi + \gamma P_\pi v_\pi$
   - Przeformułowanie: $(I - \gamma P_\pi) v_\pi = r_\pi$
   - Rozwiązanie: $v_\pi = (I - \gamma P_\pi)^{-1} r_\pi$

In [3]:
def build_P_r_for_policy(P, pi):
    """
    Buduje (P_pi, r_pi) dla zadanej polityki π.
    
    Parametry:
    ----------
    P : dict
        Model świata P[s][a] -> [(p, s', r, terminated), ...]
    pi : ndarray (nS, nA)
        Polityka: π[s,a] = P(A_t = a | S_t = s)
    
    Zwraca:
    -------
    P_pi : ndarray (nS, nS)
        Macierz przejść dla polityki π
    r_pi : ndarray (nS,)
        Wektor nagród oczekiwanych dla polityki π
    
    Wyjaśnienie:
    -----------
    Z ogólnego MDP (P[s][a]) robimy "świat widziany przez politykę π"
    poprzez uśrednianie po akcjach wybranych przez politykę.
    """
    nS = len(P)
    nA = len(P[0])
    
    P_pi = np.zeros((nS, nS), dtype=float)
    r_pi = np.zeros(nS, dtype=float)
    
    for s in range(nS):
        for a in range(nA):
            w = float(pi[s, a])  # Waga akcji a wg polityki π
            
            if w == 0.0:
                continue  # Polityka nigdy nie wybiera tej akcji
            
            outcomes = P[s][a]
            if not outcomes:
                continue  # Akcja niedostępna
            
            # Sumujemy po wszystkich możliwych skutkach akcji
            for (p, s2, r, terminated) in outcomes:
                # Nagroda oczekiwana
                r_pi[s] += w * p * float(r)
                
                # Przejścia (pomijamy stany terminalne)
                if not terminated:
                    P_pi[s, int(s2)] += w * p
    
    return P_pi, r_pi


# ========== A1.2: Ewaluacja kilku polityk ==========

print("\n" + "=" * 70)
print("A1.2: EWALUACJA POLITYKI - Porównanie różnych strategii")
print("=" * 70)

gamma = 0.9  # Współczynnik dyskontowania
P, nS, nA = build_recycling_robot_P()

H, L = 0, 1
SEARCH, WAIT, RECHARGE = 0, 1, 2

# ===== POLITYKA 1: Search w H, Recharge w L =====
print("\n1. POLITYKA π₁: Szukaj w H, Ładuj w L")
print("   Logika: Szukamy gdy mamy energię, ładujemy gdy kończy się")

pi1 = np.zeros((nS, nA))
pi1[H, SEARCH] = 1.0
pi1[L, RECHARGE] = 1.0

P_pi1, r_pi1 = build_P_r_for_policy(P, pi1)
v1 = evaluate_policy_linear_system(P_pi1, r_pi1, gamma)

print(f"   v_π₁(H) = {v1[0]:.4f}  (wartość w stanie wysokiej energii)")
print(f"   v_π₁(L) = {v1[1]:.4f}  (wartość w stanie niskiej energii)")
print(f"   Średnia: {np.mean(v1):.4f}")

# ===== POLITYKA 2: Wait wszędzie =====
print("\n2. POLITYKA π₂: Czekaj wszędzie")
print("   Logika: Minimalizujemy ryzyko - nigdy nie szukamy")

pi2 = np.zeros((nS, nA))
pi2[H, WAIT] = 1.0
pi2[L, WAIT] = 1.0

P_pi2, r_pi2 = build_P_r_for_policy(P, pi2)
v2 = evaluate_policy_linear_system(P_pi2, r_pi2, gamma)

print(f"   v_π₂(H) = {v2[0]:.4f}")
print(f"   v_π₂(L) = {v2[1]:.4f}")
print(f"   Średnia: {np.mean(v2):.4f}")

# ===== POLITYKA 3: Search wszędzie =====
print("\n3. POLITYKA π₃: Szukaj wszędzie")
print("   Logika: Maksymalizujemy zbieranie puszek - zawsze szukamy")

pi3 = np.zeros((nS, nA))
pi3[H, SEARCH] = 1.0
pi3[L, SEARCH] = 1.0  # W L może dojść do rozładowania

P_pi3, r_pi3 = build_P_r_for_policy(P, pi3)
v3 = evaluate_policy_linear_system(P_pi3, r_pi3, gamma)

print(f"   v_π₃(H) = {v3[0]:.4f}")
print(f"   v_π₃(L) = {v3[1]:.4f}")
print(f"   Średnia: {np.mean(v3):.4f}")

# ===== PORÓWNANIE =====
print("\n" + "-" * 70)
print("PORÓWNANIE POLITYK:")
print("-" * 70)

data = {
    'Polityka': ['π₁: Search-H, Recharge-L', 'π₂: Wait wszędzie', 'π₃: Search wszędzie'],
    'v(H)': [f"{v1[0]:.4f}", f"{v2[0]:.4f}", f"{v3[0]:.4f}"],
    'v(L)': [f"{v1[1]:.4f}", f"{v2[1]:.4f}", f"{v3[1]:.4f}"],
    'Średnia': [f"{np.mean(v1):.4f}", f"{np.mean(v2):.4f}", f"{np.mean(v3):.4f}"]
}

df = pd.DataFrame(data)
print(df.to_string(index=False))

# Znalezienie najlepszej polityki
best_idx = np.argmax([np.mean(v1), np.mean(v2), np.mean(v3)])
policies_names = ['π₁', 'π₂', 'π₃']
print(f"\n✓ NAJLEPSZA polityka: {policies_names[best_idx]}")
print(f"  Uzasadnienie: Maksymalna średnia wartość = {[np.mean(v1), np.mean(v2), np.mean(v3)][best_idx]:.4f}")

print("\nWNIOSEK:")
print("-" * 70)
print("• π₁ balansuję między bezpieczeństwem (RECHARGE w L) a zyskiem (SEARCH w H)")
print("• π₂ jest konserwatyjna - traci potencjał z SEARCH")
print("• π₃ jest agresywna - ale może czasem się rozładować w L")
print("• Dla domyślnych parametrów π₁ osiąga najlepszy wynik!")


A1.2: EWALUACJA POLITYKI - Porównanie różnych strategii

1. POLITYKA π₁: Szukaj w H, Ładuj w L
   Logika: Szukamy gdy mamy energię, ładujemy gdy kończy się
   v_π₁(H) = 42.3729  (wartość w stanie wysokiej energii)
   v_π₁(L) = 38.1356  (wartość w stanie niskiej energii)
   Średnia: 40.2542

2. POLITYKA π₂: Czekaj wszędzie
   Logika: Minimalizujemy ryzyko - nigdy nie szukamy
   v_π₂(H) = 10.0000
   v_π₂(L) = 10.0000
   Średnia: 10.0000

3. POLITYKA π₃: Szukaj wszędzie
   Logika: Maksymalizujemy zbieranie puszek - zawsze szukamy
   v_π₃(H) = 39.4634
   v_π₃(L) = 33.6098
   Średnia: 36.5366

----------------------------------------------------------------------
PORÓWNANIE POLITYK:
----------------------------------------------------------------------
                Polityka    v(H)    v(L) Średnia
π₁: Search-H, Recharge-L 42.3729 38.1356 40.2542
       π₂: Wait wszędzie 10.0000 10.0000 10.0000
     π₃: Search wszędzie 39.4634 33.6098 36.5366

✓ NAJLEPSZA polityka: π₁
  Uzasadnienie: Mak

---

## Sekcja 5: Optymalizacja - Znajdowanie π* (A1.3)

### Cel:
Znaleźć **optymalną politykę** $\pi_*$ sprawdzając wszystkie możliwe deterministyczne strategie.

### Dlaczego brute force?
- Robot ma tylko 2 stany
- W każdym stanie max 3 akcje  
- Liczba wszystkich polityk: 2 × 3 = **6 możliwości**
- Możemy sprawdzić wszystkie i wybrać najlepszą!

In [4]:
def all_deterministic_policies_robot():
    """
    Generuje wszystkie możliwe deterministyczne polityki dla robota.
    
    Zwraca:
    -------
    policies : list
        Lista wszystkich 6 polityk, każda jako macierz (2, 3)
    """
    H, L = 0, 1
    SEARCH, WAIT, RECHARGE = 0, 1, 2
    
    policies = []
    
    # Dla każdej kombinacji akcji w H i L
    for aH in [SEARCH, WAIT]:
        for aL in [SEARCH, WAIT, RECHARGE]:
            pi = np.zeros((2, 3))
            pi[H, aH] = 1.0
            pi[L, aL] = 1.0
            policies.append(pi)
    
    return policies


def best_policy_robot(P, gamma=0.9):
    """
    Znajduje optymalną politykę poprzez brute force.
    
    Parametry:
    ----------
    P : dict
        Model MDP
    gamma : float
        Współczynnik dyskontowania
    
    Zwraca:
    -------
    best_pi : ndarray
        Optymalna polityka
    best_vH : float
        Wartość w stanie H dla optymalnej polityki
    """
    best_pi = None
    best_vH = None
    
    for pi in all_deterministic_policies_robot():
        P_pi, r_pi = build_P_r_for_policy(P, pi)
        v = evaluate_policy_linear_system(P_pi, r_pi, gamma)
        vH = float(v[0])
        
        if best_vH is None or vH > best_vH:
            best_vH = vH
            best_pi = pi
    
    return best_pi, best_vH


# ========== A1.3: Znajdowanie π* ==========

print("\n" + "=" * 70)
print("A1.3: OPTYMALIZACJA - Znajdowanie najlepszej polityki π*")
print("=" * 70)

P, nS, nA = build_recycling_robot_P()
action_name = {0: "SEARCH", 1: "WAIT", 2: "RECHARGE"}

# Znalezienie π*
pi_star, vH_star = best_policy_robot(P, gamma=0.9)

print(f"\nOptymalna polityka π* dla domyślnych parametrów:")
print(f"  Parametry: α=0.8, β=0.4, r_search=5.0, r_wait=1.0, rescue_cost=-3.0, γ=0.9")
print(f"\n  π*(H) = {action_name[int(np.argmax(pi_star[0]))]}")
print(f"  π*(L) = {action_name[int(np.argmax(pi_star[1]))]}")
print(f"\n  Wartość: v_π*(H) = {vH_star:.4f}")

# Obliczenie v* dla obu stanów
P_pi_star, r_pi_star = build_P_r_for_policy(P, pi_star)
v_star = evaluate_policy_linear_system(P_pi_star, r_pi_star, 0.9)

print(f"  Wartość: v_π*(L) = {v_star[1]:.4f}")

print("\nWNIOSEK:")
print("-" * 70)
print("• Optymalnie jest szukać (SEARCH) gdy mamy energię (stan H)")
print("• Optymalnie jest ładować (RECHARGE) gdy energia się kończy (stan L)")
print("• Ta polityka osiąga najlepszy kompromis między zyskiem a bezpieczeństwem")


A1.3: OPTYMALIZACJA - Znajdowanie najlepszej polityki π*

Optymalna polityka π* dla domyślnych parametrów:
  Parametry: α=0.8, β=0.4, r_search=5.0, r_wait=1.0, rescue_cost=-3.0, γ=0.9

  π*(H) = SEARCH
  π*(L) = RECHARGE

  Wartość: v_π*(H) = 42.3729
  Wartość: v_π*(L) = 38.1356

WNIOSEK:
----------------------------------------------------------------------
• Optymalnie jest szukać (SEARCH) gdy mamy energię (stan H)
• Optymalnie jest ładować (RECHARGE) gdy energia się kończy (stan L)
• Ta polityka osiąga najlepszy kompromis między zyskiem a bezpieczeństwem


---

## Sekcja 6: Analiza Wrażliwości - Sweep Parametrów (A1.4)

### KLUCZOWA OBSERWACJA:

**Parametry środowiska (α, β, rescue_cost, γ) są CZĘŚCIĄ ŚWIATA, nie polityki.**

Ale wpływają na Q-wartości akcji, co zmienia **którą politykę system powinien wybrać**!

### Eksperyment:

Utrzymując stałe:
- `α = 0.8` (SEARCH w H jest bezpieczny)
- `r_search = 5.0, r_wait = 1.0`
- `γ = 0.9`

Zmieniamy:
- **`β`** (od 0.1 do 0.9): jak bezpieczne jest SEARCH w stanie L?
- **`rescue_cost`** (od -1.0 do -10.0): jak wielka jest kara za rozładowanie?

Obserwujemy: **Kiedy zmienia się π*(L)?**

In [5]:
def run_sweep(beta_list=None, rescue_list=None, alpha=0.8, r_search=5.0, r_wait=1.0, gamma=0.9):
    """
    Przeprowadza sweep parametrów środowiska i wyznacza π* dla każdej kombinacji.
    """
    if beta_list is None:
        beta_list = [0.1, 0.3, 0.5, 0.7, 0.9]
    if rescue_list is None:
        rescue_list = [-1.0, -3.0, -6.0, -10.0]
    
    action_name = {0: "SEARCH", 1: "WAIT", 2: "RECHARGE"}
    results = []
    
    for beta in beta_list:
        for rescue_cost in rescue_list:
            P, _, _ = build_recycling_robot_P(
                alpha=alpha, beta=beta,
                r_search=r_search, r_wait=r_wait,
                rescue_cost=rescue_cost
            )
            pi_star, _ = best_policy_robot(P, gamma=gamma)
            aH = action_name[int(np.argmax(pi_star[0]))]
            aL = action_name[int(np.argmax(pi_star[1]))]
            results.append({
                'beta': beta,
                'rescue_cost': rescue_cost,
                'π*(H)': aH,
                'π*(L)': aL
            })
    
    return pd.DataFrame(results)


# ========== A1.4: SWEEP PARAMETRÓW ==========

print("\n" + "=" * 70)
print("A1.4: SWEEP PARAMETRÓW - Wrażliwość na zmiany środowiska")
print("=" * 70)

print("\nUstawienia STAŁE:")
print("  α = 0.8      (SEARCH w H jest bezpieczny)")
print("  r_search = 5.0")
print("  r_wait = 1.0")
print("  γ = 0.9      (dyskontowanie)")

print("\nUstawienia ZMIENNE:")
print("  β: [0.1, 0.3, 0.5, 0.7, 0.9]      (bezpieczeństwo SEARCH w L)")
print("  rescue_cost: [-1.0, -3.0, -6.0, -10.0]  (kara za rozładowanie)")

# Przeprowadzenie sweepów
df_sweep = run_sweep()

print("\n" + "-" * 70)
print("WYNIKI SWEEPÓW:")
print("-" * 70)
print(df_sweep.to_string(index=False))

# Analiza: Kiedy zmienia się π*(L)?
print("\n" + "-" * 70)
print("ANALIZA ZMIAN π*(L):")
print("-" * 70)

transitions = []
for beta in [0.1, 0.3, 0.5, 0.7, 0.9]:
    subset = df_sweep[df_sweep['beta'] == beta]
    
    # Jak się zmienia π*(L) wraz ze wzrostem kary?
    actions_for_beta = subset['π*(L)'].unique()
    
    if len(actions_for_beta) > 1:
        print(f"\nβ = {beta}:")
        for rescue in subset['rescue_cost'].unique():
            action = subset[subset['rescue_cost'] == rescue]['π*(L)'].values[0]
            print(f"  rescue_cost = {rescue:6.1f} → π*(L) = {action}")
    else:
        action = actions_for_beta[0]
        print(f"\nβ = {beta}: π*(L) zawsze = {action}")

# Wniosek
print("\n" + "=" * 70)
print("WNIOSKI:")
print("=" * 70)
print("""
1. π*(H) ZAWSZE = SEARCH
   → Gdy energia wysoka, zawsze opłaca się szukać
   
2. π*(L) ZMIENIA się w zależności od PARAMETRÓW:
   
   a) Przy MAŁYM β (SEARCH w L jest RYZYKOWNY):
      → Dominuje RECHARGE (bezpieczne ładowanie)
      → Robot wolej ładować niż ryzykować rozładowanie
   
   b) Przy DUŻYM β (SEARCH w L jest BEZPIECZNY):
      → Dominuje SEARCH (zbieranie puszek)
      → Robot wie że się nie rozładuje, więc szuka
   
   c) Przy MAŁYM rescue_cost (kara -1.0):
      → Większa skłonność do SEARCH w L
      → Mała kara = można ryzykować
   
   d) Przy DUŻYM rescue_cost (kara -10.0):
      → Większa skłonność do RECHARGE w L
      → Duża kara = lepiej być bezpiecznym

3. PRÓG DECYZYJNY:
   → Istnieje punkt przejścia (dla β ≈ 0.5-0.7)
   → Gdzie π*(L) zmienia się z RECHARGE na SEARCH
   → Dokładna wartość zależy od rescue_cost
""")


A1.4: SWEEP PARAMETRÓW - Wrażliwość na zmiany środowiska

Ustawienia STAŁE:
  α = 0.8      (SEARCH w H jest bezpieczny)
  r_search = 5.0
  r_wait = 1.0
  γ = 0.9      (dyskontowanie)

Ustawienia ZMIENNE:
  β: [0.1, 0.3, 0.5, 0.7, 0.9]      (bezpieczeństwo SEARCH w L)
  rescue_cost: [-1.0, -3.0, -6.0, -10.0]  (kara za rozładowanie)

----------------------------------------------------------------------
WYNIKI SWEEPÓW:
----------------------------------------------------------------------
 beta  rescue_cost  π*(H)    π*(L)
  0.1         -1.0 SEARCH RECHARGE
  0.1         -3.0 SEARCH RECHARGE
  0.1         -6.0 SEARCH RECHARGE
  0.1        -10.0 SEARCH RECHARGE
  0.3         -1.0 SEARCH RECHARGE
  0.3         -3.0 SEARCH RECHARGE
  0.3         -6.0 SEARCH RECHARGE
  0.3        -10.0 SEARCH RECHARGE
  0.5         -1.0 SEARCH   SEARCH
  0.5         -3.0 SEARCH RECHARGE
  0.5         -6.0 SEARCH RECHARGE
  0.5        -10.0 SEARCH RECHARGE
  0.7         -1.0 SEARCH   SEARCH
  0.7         -3.

---

## Sekcja 7: Dodatkowe Eksperymenty - Wpływ Gamma na π*

### Pytanie:
Jak współczynnik dyskontowania (γ) wpływa na optymalną politykę?

**Intuicja:**
- γ niskie (0.5): Agent myśli krótkoterminowo → preferuje szybkie nagrody
- γ wysokie (0.99): Agent myśli długoterminowo → bardziej ostrożny, planuje na przyszłość

In [6]:
print("\n" + "=" * 70)
print("EKSPERYMENT: Wpływ Gamma (dyskontowania) na π*")
print("=" * 70)

gamma_values = [0.5, 0.9, 0.99]
beta_test = [0.3, 0.7, 0.9]
rescue_test = [-3.0, -10.0]

for gamma_test in gamma_values:
    print(f"\n{'─' * 70}")
    print(f"GAMMA = {gamma_test:0.2f}")
    print(f"{'─' * 70}")
    
    results_gamma = []
    
    for beta in beta_test:
        for rescue_cost in rescue_test:
            P, _, _ = build_recycling_robot_P(
                alpha=0.8, beta=beta,
                r_search=5.0, r_wait=1.0,
                rescue_cost=rescue_cost
            )
            pi_star, vH = best_policy_robot(P, gamma=gamma_test)
            
            aH = "SEARCH" if int(np.argmax(pi_star[0])) == 0 else "WAIT"
            aL_idx = int(np.argmax(pi_star[1]))
            aL = ["SEARCH", "WAIT", "RECHARGE"][aL_idx]
            
            results_gamma.append({
                'β': f"{beta:.1f}",
                'rescue': f"{rescue_cost:.0f}",
                'π*(H)': aH,
                'π*(L)': aL,
                'v(H)': f"{vH:.2f}"
            })
    
    df_gamma = pd.DataFrame(results_gamma)
    print(df_gamma.to_string(index=False))

print("\n" + "=" * 70)
print("OBSERWACJE:")
print("=" * 70)
print("""
• γ = 0.50 (myślenie krótkoterminowe):
  - Preferuje natychmiastowe nagrody
  - Może być bardziej agresywny w L (SEARCH)
  
• γ = 0.90 (balans):
  - Równowaga między teraźniejszością a przyszłością
  - Standartowy wybór w wielu aplikacjach
  
• γ = 0.99 (myślenie długoterminowe):
  - Zdyskontowane nagrody przyszłych kroków są ważne
  - Może być bardziej konserwatywny (RECHARGE)
  - Agent bardziej planuje na przyszłość
""")


EKSPERYMENT: Wpływ Gamma (dyskontowania) na π*

──────────────────────────────────────────────────────────────────────
GAMMA = 0.50
──────────────────────────────────────────────────────────────────────
  β rescue  π*(H)    π*(L) v(H)
0.3     -3 SEARCH RECHARGE 9.09
0.3    -10 SEARCH RECHARGE 9.09
0.7     -3 SEARCH   SEARCH 9.36
0.7    -10 SEARCH RECHARGE 9.09
0.9     -3 SEARCH   SEARCH 9.75
0.9    -10 SEARCH   SEARCH 9.54

──────────────────────────────────────────────────────────────────────
GAMMA = 0.90
──────────────────────────────────────────────────────────────────────
  β rescue  π*(H)    π*(L)  v(H)
0.3     -3 SEARCH RECHARGE 42.37
0.3    -10 SEARCH RECHARGE 42.37
0.7     -3 SEARCH RECHARGE 42.37
0.7    -10 SEARCH RECHARGE 42.37
0.9     -3 SEARCH   SEARCH 46.11
0.9    -10 SEARCH   SEARCH 42.70

──────────────────────────────────────────────────────────────────────
GAMMA = 0.99
──────────────────────────────────────────────────────────────────────
  β rescue  π*(H)    π*(L)   

## Sekcja 8: Podsumowanie i Wnioski


###  STRUKTURA MDP

- **Model P[s][a]**: opisuje dynamikę świata (przejścia, nagrody)
- **Polityka π[s,a]**: opisuje zachowanie agenta (wybór akcji)  
- **Funkcja wartości v_π[s]**: mierzy jakość polityki

**Wniosek**: Te trzy komponenty są **niezależne** ale **powiązane**!

### EWALUACJA POLITYKI (A1.2)

* Dla zadanej polityki możemy obliczyć dokładnie $v_\pi$  
* Rozwiązując układ równań liniowych: $(I - \gamma P_\pi)v = r_\pi$  
* Pozwala porównywać różne strategie


### OPTYMALIZACJA (A1.3)

* Polityka optymalna $\pi_*$ maksymalizuje wartość oczekiwaną  
* Dla małych MDP możemy znaleźć $\pi_*$ brute force'em  
* Dla dużych MDP używamy algorytmów DP (rozdz. 4)


### WRAŻLIWOŚĆ NA PARAMETRY (A1.4)

* Parametry środowiska (α, β, rescue_cost, γ) wpływają na $\pi_*$  
* Istnieją **progi decyzyjne** - gdzie $\pi_*$ zmienia się skokowo  
* Ta sama struktura problemu może mieć **różne π*** dla różnych środowisk


### PRAKTYCZNE IMPLIKACJE

* Ważne jest rozumieć **jakie są parametry naszego świata**  
* Nie wszystkie strategie są optymalne w każdym środowisku  
* **Dobry agent** musi się dostosowywać do zmian otoczenia
